> Upgrade Kernel For Debian

Intel的Skylake 6500 CPU自带的HD 530显卡，新版本内核应该自带驱动了，但是安装的Debian 8 Jessie使用的还是3.16的旧版本内核，所以很多驱动都不行，需要为Debian更新驱动，否则无法识别显示器，无法调整显示器分辨率。

在3.16内核下，可以看到驱动是不正确的：
```
glxinfo | grep "OpenGL vendor string"
输出结果为：
OpenGL vendor string: VMware, Inc.
```

更新到4.6内核后，可以看变换了，选择Display显示器可以选择分辨率，旋转等等操作，屏幕亮度也支持了，声音输出正常了，可以有声音了
```
i965_dri.so does not support the 0x191b PCI ID.
libGL error: failed to create dri screen
libGL error: failed to load driver: i965
OpenGL vendor string: VMware, Inc.

这里错是因为我的是intel+nvidia显卡。。。坑爹
```

需要修正，参考文章 [A](http://www.cnblogs.com/ibgo/p/4113940.html) [B](https://www.reddit.com/r/debian/comments/4adhwi/updating_linux_kernel_in_debian_jessie/)

**为Debian 8 Jessie 更新4.6内核**
***

## 1. 添加backports源(注意与Debian版本相关，我的是Jessie)
使用vim打开/etc/apt/source.list文件（注意备份），加入一下内容：
```
deb http://httpredir.debian.org/debian jessie-backports main
```

更新源文件：
```
apt-get update
```

## 2. 更新backports源
```
apt-get -t jessie-backports upgrade
```

查找新内核
```
aptitude search linux-image
```

## 3. 安装新内核
```
apt-get -t jessie-backports install linux-image-4.0.0-0.bpo.3-amd64
```

安装完后重启电脑，选择新的内核进入系统。

可以看看系统里面有多少个内核：
```
dpkg --get-selections | grep linux-image
```

如果要删除就内核：
```
apt-get remove linux-image-x.x.x.--..
```

*** 
更新完内核后Debian的驱动好了，有窗口效果了，声音需要在 右键-settings-声音-选择声音输出；然后开启system alter的声音

***
### Jupyter 修改启动后路径问题：
1. 创建空白配置文件：

    `#jupyter notebook --generate-config`
   
   这里会提示创建好的配置文件目录，使用vim编辑。

2. 修改配置文件 jupyter_notebook_config.py
    '#c.NotebookApp.notebook_dir = ... ' -> 'c.NotebookApp.notebook_dir = ...'
    
    这里放你需要的路径。

### Jupyter 启动不打开浏览器选项：
修改配置文件 jupyter_notebook_config.py

    `#c.NotebookApp.open_browser = True` -> `c.NotebookApp.open_browser = False`

### Jupyter 自启动问题
在Debian中，使用.profile来控制用户登录后启动什么程序，因此把启动Jupyter的任务放在了.profile中

内容：
```
if test $( pgrep -f jupyter-notebook | wc -l ) -eq 0
then
    echo 'Jupyter Notebook is not here, try to start it'
    /home/imse/Anaconda2/bin/jupyter notebook >/dev/null 2>&1 &
else
    echo 'Jupyter Notebook is here'
```

***
### 解决Debian默认声音不正确问题
问题描述，每次login后，Debian都把声音默认输出到了HDMI所以没有开机后没有声音，必须手动选择默认声音默认输出后才有声音，google后得到不错的解决方案: [A](https://bbs.archlinux.org/viewtopic.php?id=164868)

大概是要指定默认音频输出方式：
I solve this problem myself

Followings are my steps:

1. login and change the default output to analog

2. Run
    `# aplay -l`
    
    and find that analog is card 0

3. Run
    `# pacmd list`

    and find the line:

    active profile: <output:analog-stereo+input:analog-stereo>

    'output:analog-stereo+input:analog-stereo' is my profile name,  so add

    **set-card-profile 0 output:analog-stereo+input:analog-stereo**

    to /etc/pulse/default.pa and save

4. restart pulseaudio and found it works smile